# Rango Consumos por Persona

Se calcula con consumos por período de prestación en un reporte y stock en otro reporte

In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# Lectura df de consumos:
ruta_cons = 'Consumo_SanCor_12M/30.2 Consumo 12 M.xlsx'
dfc = pd.read_excel(ruta_cons)

In [3]:
ruta_stock = './Stock_SanCor_12M/'

In [4]:
# Levanto el stock existente en una carpeta:

ruta = ruta_stock
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfs = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfs = pd.concat([dfs, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")        


Inicio del proceso: 12:30:16
Se cargó el archivo:  30.1 Stock Brinkmann 12 M.xlsx a las: 12:30:47
Fin de la carga de archivos


In [5]:
# bkp temporal
dfsbkp = dfs
# dfs = dfsbkp

## 1.2- Renombrar columnas

In [6]:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)
dfs.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)


dfc.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo2',
                           'Consumos Costo con Otros Subsidios': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)

In [7]:
# Agregamos a dfs el consumo de dfc:
dfs = pd.merge(left = dfs, right = dfc, left_on='Persona', right_on='Persona', how = "left")

In [8]:
len(dfs)

758659

In [9]:
dfs.Stock.sum()

7190382

# 2- Cálculo de marca GC

Pasos a seguir:<br>
1- Marcar gc con la actualización de los valores acordados<br>
2- Marcar diagnosticos con la logica acordada<br>
<br>

In [10]:
# Seteo de variables para marcas:

# Defino importe gran consumidor
importe_gc = 636370

# De cada rango se define su límite inferior para consumo mensual
c_muy_alto = importe_gc / 12
c_alto = 120000 / 12
c_medio = 6000 / 12
c_bajo = 2400 / 12 # 200 pesos por mes por el efecto cápita
# El consumo c_muy_bajo queda por defecto.

In [11]:
dfs.columns

Index(['Persona', 'Stock', 'Consumo3'], dtype='object')

In [12]:
######## MARCO GRAN CONSUMIDOR:
# https://es.stackoverflow.com/questions/193267/c%C3%B3mo-creo-una-columna-en-un-dataframe-dependiendo-del-valor-de-otras-dos-column
dfs['GC'] = np.where(dfs['Consumo3'] > importe_gc, 'GC','NO GC')

######## MARCO RANGO CONSUMOS:
# Cálculo de consumo anualizado para rango consumos:
dfs['Cons_mensualizado'] = (dfs['Consumo3'] / dfs['Stock'])

# Como existen personas con stock == 0 -> reemplazo los nan con el valor de la columna "Consumo3"
# No encontré una función -> hago una reimputación manual
for i in range(len(dfs)):
    if np.isnan(dfs["Cons_mensualizado"][i]) == True:
        dfs["Cons_mensualizado"][i] = dfs["Consumo3"][i]

# Lista de condiciones para selección
# https://www.delftstack.com/es/howto/python-pandas/how-to-create-dataframe-column-based-on-given-condition-in-pandas/#lista-de-comprensi%25C3%25B3n-para-crear-nuevas-columnas-dataframe-basadas-en-una-condici%25C3%25B3n-dada-en-pandas
# Método np.select()
condiciones = [
    (dfs['Cons_mensualizado'] >= c_muy_alto) ,
    (dfs['Cons_mensualizado'] >= c_alto) ,
    (dfs['Cons_mensualizado'] >= c_medio) ,
    (dfs['Cons_mensualizado'] >= c_bajo) ,
    (dfs['Cons_mensualizado'] < c_bajo)]

# Lista de resultados en función de las selecciones
valores_condic = ['Muy Alto', 'Alto', 'Medio', 'Bajo', 'Muy Bajo']     

# Generación de la columna calculada
dfs['Rango_Cons'] = np.select(condiciones, valores_condic, default='Muy Bajo') # se analizaron y surgen de consumo 3 == 0

# debería eliminar dfc de memoria porque no se usa más
# debería eliminar dfsbkp de memoria porque no se usa más

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [13]:
# Exportamos la auxiliar por persona para entender marcas rango consumos
dfs.to_csv("Rango_cons.csv", encoding = 'latin1', 
                decimal=',', index = False)

In [14]:
# finaliza el script

# 3- Calcular marcas de diagnóstico -- NO SE REALIZA MAS AQUI

El procedimiento a seguir debe ser el siguiente:<br>
1- Si el 100% del tiempo tuvo un diagnóstico va ese. <br>
2- Si más del 60% tuvo algún diagnóstico va ese.<br>
3- Si no supera 60% en algún clúster se lo define como su peor situación (con diagnóstico).<br>
4- (solamente para consumos) = si no se encuentra la persona en el tiempo de stock analizado se lo marca como "No Corresponde".<br>

In [ ]:
# dfs2 es el 2do dataframe que usamos para hacer el pegado de diag.
ruta_dfs2 = 'C:\\Users\\lsaucedo\\Desktop\\Brinkmann v2\\Stock_Brinkmann/'
dfs2 = pd.read_excel(ruta_dfs2 + "Stock_brinkmann_202010-202109.xlsx")

In [ ]:
dfs2.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)

In [ ]:
# Ruta de auxiliares
ruta_aux = 'C:\\modelos\\Auxiliares\\'

In [ ]:
# Definiciones:
# Proporción mínima de meses con diagnóstico para ser considerado con algún diagnóstico:
prop_limite = 0.4
# Se puede agregar los campos considerados como diag..

# Reemplazo valores de filas con replace:
dfs2['Diabetes'] = dfs2['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs2['Hepatitis'] = dfs2['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs2['PMI'] = dfs2['PMI'].replace(['NO', 'SI'], [0, 1])
dfs2['Cronico'] = dfs2['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs2['Oncologico'] = dfs2['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs2['GC'] = dfs2['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs2['Discapacidad'] = dfs2['Discapacidad'].replace(['N', 'S'], [0, 1])

# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
# Se tendría que mejorar para generalizar:
suma = (dfs2['Diabetes'] + dfs2['Hepatitis'] + dfs2['PMI'] + 
        dfs2['Cronico'] + dfs2['Oncologico'] + dfs2['GC'] + dfs2['Discapacidad'])
dfs2['algun_diag'] = np.where(suma > 0, 1, 0)

# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs2.pivot_table(columnas , filas, aggfunc = np.sum))


# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [ ]:
# tener en cuenta que al agregar la pos. asoc. a la td pueden aparecer
#personas duplicadas si se exportan los reportes en diferente momento..

In [ ]:
# lo que se acorta el df tiene relacion con personas duplicadas.
# en el caso de stock continúa ok.
len(aux_diag)

In [ ]:
aux_diag.Stock.sum()

# Exporto la info a csv

In [ ]:
aux_diag.to_csv("Dx_costeos.csv", encoding = 'latin1', 
                decimal=',', index = False)